In [1]:
import datetime
import random
import json
import dotenv
from azure.iot.device import IoTHubDeviceClient
from azure.iot.hub import IoTHubRegistryManager
from os import getenv

In [2]:
DEVICE_CONNECTION_STRING = "HostName=my-project-iot-hub.azure-devices.net;DeviceId=1;SharedAccessKey=MerqNqhUPoaOMGeM18VijAnc5Ly87ndqLAIoTI/QV2s="
device_id = "1"

In [3]:
dotenv.load_dotenv("./device-function-app/.env")
# pRINT OUT THE ENVIRONMENT VARIABLES
getenv("IOT_HUB_CONNECTION_STRING")

'HostName=my-project-iot-hub.azure-devices.net;SharedAccessKeyName=iothubowner;SharedAccessKey=5aQcNZpJd+8RJOr5lo2ORq4Ge2ex/Bl9BAIoTIdR7cQ='

In [4]:
registry = IoTHubRegistryManager.from_connection_string(getenv("IOT_HUB_CONNECTION_STRING"))
device_twin = registry.get_twin(device_id)
# Print the twin
device_type_id = device_twin.tags["deviceTypeId"]
oid = "XYZZ"
device_type_id

'1'

In [5]:
# Create an IoT Hub client
device = IoTHubDeviceClient.create_from_connection_string(DEVICE_CONNECTION_STRING)
device.connect()

In [6]:
from abc import ABC
class Measurement(ABC):
    def __init__(self, device_type_id, oid: str, datetime: str):
        self.device_type_id = device_type_id
        self.oid = oid
        self.datetime = datetime

In [7]:
class PedometerMeasurement(Measurement):
    def __init__(self, steps, distance, datetime: datetime.datetime, device_type_id, oid: str, unit="30m"):
        super().__init__(device_type_id, oid, datetime.isoformat())
        self.steps = steps
        self.distance = distance
        self.unit = unit
    
    def __str__(self):
        return f"Device Type: {self.device_type_id}, Steps: {self.steps}, Distance: {self.distance}, Unit: {self.unit}, Datetime: {self.datetime}"

In [8]:
class ScaleMeasurement(Measurement):
    def __init__(self, weight, timestamp: datetime.datetime, device_type_id, oid: str, unit="kg"):
        super().__init__(device_type_id, oid, timestamp.isoformat())
        self.weight = weight
        self.unit = unit

    def __str__(self):
        return f"Device Type: {self.device_type_id}, Weight: {self.weight}, Unit: {self.unit}, Datetime: {self.datetime}"

In [9]:
current_time = datetime.datetime.now()
current_time -= datetime.timedelta(minutes=current_time.minute % 30, seconds=current_time.second, microseconds=current_time.microsecond)
thirty_minutes = datetime.timedelta(minutes=30) # Round to the previous whole 30 minutes
# Send pedometer measurements
for i in range(10):
    pm = PedometerMeasurement(random.randint(0, 100), random.randint(0, 100), current_time, device_type_id, oid)
    print(pm)
    current_time += thirty_minutes
    message = json.dumps(pm.__dict__)
    print(message)
    device.send_message(message)

Device Type: 1, Steps: 71, Distance: 99, Unit: 30m, Datetime: 2024-02-03T18:30:00
{"device_type_id": "1", "oid": "XYZZ", "datetime": "2024-02-03T18:30:00", "steps": 71, "distance": 99, "unit": "30m"}
Device Type: 1, Steps: 85, Distance: 43, Unit: 30m, Datetime: 2024-02-03T19:00:00
{"device_type_id": "1", "oid": "XYZZ", "datetime": "2024-02-03T19:00:00", "steps": 85, "distance": 43, "unit": "30m"}
Device Type: 1, Steps: 54, Distance: 64, Unit: 30m, Datetime: 2024-02-03T19:30:00
{"device_type_id": "1", "oid": "XYZZ", "datetime": "2024-02-03T19:30:00", "steps": 54, "distance": 64, "unit": "30m"}
Device Type: 1, Steps: 74, Distance: 94, Unit: 30m, Datetime: 2024-02-03T20:00:00
{"device_type_id": "1", "oid": "XYZZ", "datetime": "2024-02-03T20:00:00", "steps": 74, "distance": 94, "unit": "30m"}
Device Type: 1, Steps: 77, Distance: 39, Unit: 30m, Datetime: 2024-02-03T20:30:00
{"device_type_id": "1", "oid": "XYZZ", "datetime": "2024-02-03T20:30:00", "steps": 77, "distance": 39, "unit": "30m"}
